# 查询etf的对应指数

In [1]:
from jqdata import *
import pandas as pd
import numpy as np

def calc_corr_coef(index_a, index_b, start_dt, end_dt):
    return np.corrcoef(
        get_price(index_a, start_dt, end_dt, fields='close')['close'].values,
        get_price(index_b, start_dt, end_dt, fields='close')['close'].values
    )[0, 1]

def get_corr_coef_index(stock_base, days=30, now_date=datetime.datetime.now()):
    # type: (str, int, Union[datetime.datetime, datetime.date, str]) -> Optional[str]

    trade_days = get_trade_days(end_date=now_date, count=days)
    start_dt = trade_days[0]
    end_dt = trade_days[-1]
    index_list = list(get_all_securities(types=['index'], date=now_date).index)

    dict_corr = {
        index:
            {'name': get_security_info(index).display_name,
             'corrcoef': calc_corr_coef(stock_base, index, start_dt, end_dt)
             }
        for index in index_list
    }
    df_corr = pd.DataFrame.from_dict(dict_corr, orient='index').dropna()
    df_corr.sort_values(by='corrcoef', ascending=False, inplace=True)

    if len(df_corr) > 0 and df_corr.corrcoef[0] >= 0.98:
        print('%s，%s 对应标的指数==> %s, %s, 跟踪准确度: %.4f' % (
            stock_base, get_security_info(stock_base).display_name,
            df_corr.index[0], df_corr.name[0],
            df_corr.corrcoef[0]))
        #
        ret_val = df_corr.index[0]
    else:
        print('%s，%s ===> 未找到相关指数' % (
            stock_base, get_security_info(stock_base).display_name))
        #
        ret_val = None
    #
    return  # ret_val

In [34]:
get_corr_coef_index('513020.XSHG')

513020.XSHG，科技HK ===> 未找到相关指数
